# Proyecto final de Técnicas de Recogida de Datos.

MBDS Universidad de Navarra 2023/2024

Paula Sanjuan Campos


## 1. Introducción

En este proyecto se obtendrán datos de artículos científicos de la base de datos *PubMed* a través de su API. *PubMed* es una base de datos líder en la literatura biomédica y de ciencias de la salud, ofrece una gran colección de artículos, libros, revisiones y otros recursos. *PubMed* fue desarrollado y mantenido por el Centro Nacional de Información Biotecnológica (NCBI). A través del proyecto se recopila información detallada sobre artículos científicos (título, palabras claves, fecha de publicación, autores). 

Se emplea la API de *Entrez* proporcionada por la Biblioteca Nacional de Medicina de EEUU (NLM). Esta API permite acceder a la base de datos de *PubMed*, realizar consultas avanzadas y recuperar información detallada sobre artículos científicos. La API de *Entrez* ofrece una amplia gama de funciones que permiten buscar artículos por términos de búsqueda, filtros, identificadores de artículo, recuperar información específica de artículos, etc.  Las *E-utilities* deE *Entrez* utilizan una sintaxis de URL fija que traduce un conjunto estándar de parámetros de entrada en los valores necesarios para la búsqueda y recuperación de los datos solicitados. 

Como se demuestra en el código de este proyecto, a través de *Entrez* se tiene acceso a 38 bases de datos que cubren una variedad de datos biomédicos, incluidas secuencias de nucleótidos y proteínas, registros genéticos, estructuras moleculares tridimensionales y literatura biomédica. Este proyecto se centra en la última de ellas, la base de datos de literatura biomédica.

Entre las pautas de uso y requisitos para las utilidades de NCBI, destacan que hay un límite de tres solicitudes por segundo. Para mejorar el acceso se requiere una clave API que se puede solicitar en la página de configuración de la cuenta NCBI. Con la calve API se puede acceder a 10 solicitudes por segundo. Por otro lado, para minimizar el número de solicitudes y por lo tanto ser eficiente en la obtención de datos, sobre todo cuando se involucran una gran cantidad de registros, se recomienda usar el Historial de *Entrez*. Con esta herramienta, en lugar de enviar solicitudes individuales para cada registro, los usuarios pueden realizar una sola solicitud para un conjunto de registros y luego trabajar con esos datos de manera iterativa o en lotes. Esto es especialmente útil en el caso de trabajar con miles o incluso millones de registros, ya que reduce significativamente el número de solicitudes individuales necesarias y el tiempo necesario para completar la tarea. Posteriormente, se explica cómo habría que realizar el procedimiento, aunque para este proyecto no he sacado tantos registros, por lo que no lo he utilizado.

Por otro lado, otro aspecto importante es la gestión de posibles bloqueos de IP al utilizar los servicios de *Entrez*, para ello, es esecial registrar un correo electrónico. NCBI implementa este proceso para garantizar un uso responsable y equitativo de sus recursos, así como para mantener la integridad y disponibilidad de sus servicios.

Una limitación del sistema es que estas utilidades solo son capaces de recuperar datos que ya están en *Entrez*, aunque la mayoría de datos del NCBI lo están. De hecho, la búsqueda que se realiza en el siguiente código recoger artículos del último año y como se puede comprobar en el dataframe, se pueden descargar contenidos de este mismo mes. 

Respecto al funcionamiento del sistema, los registros de datos que contiene cada base de datos de *Entrez* se reconocen mediante un ID entero llamado UID (identificador único). El núcleo del sistema consiste en la realización de dos tareas: reunir la lista de UID que coincidan con una consulta de texto y recuperar un breve registro resumido llamado Resumen de documento (*DocSum*) para cada UID.


El acceso a esta API se realiza a través de la biblioteca de *Biopython*.



### Documetación de la API de Biopython

Documentación de la API de *Biopython*: https://biopython.org/docs/latest/api/index.html

Esta API es usada no solo para acceder a datos de artículos científicos, si no también con fines bioinformáticos. El paquete que se usa en el proyecto es ***Bio.Entrez***, concretamente se accede a la base de datos de *PubMed* (literatura biomédica), aunque otras opciones son: *GenBank* (secuencias géneticas), *BLAST* (análisis de secuencias biológicas). 

Ya que es una API para bioinformática, el resto de paquetes de la API tienen funcionalidades relacionadas con la obtención y manipulación de secuencias de ADN, ARN (*Bio.Seq*), para trabajar con la estructura de las proteínas (*Bio.PDB*), obtener representaciones gráficas de datos biológicos (*Bio.Graphics*), entre otros.

Las utilidades del paquete de ***Bio.Entrez*** se explican en: 
- https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4
- https://biopython.org/docs/latest/api/Bio.Entrez.html

## 2. Librerías y módulos

In [1]:
!pip install biopython
# pip install --upgrade biopython
# pip uninstall biopython


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Biopython requiere NumPy (pero se instala automáticamente con Biopython). 

In [2]:
from Bio import Entrez
import pandas as pd

Para utilizar la API, se requiere especificar una dirección de correo para la request: `Entrez.email = 'A.N.Other@example.com`. Su propósito es gestionar las solicitudes para garantizar un funcionamiento eficiente y para asegurar el cumplimiento de políticas del uso de la API.  

In [3]:
Entrez.email = "sanjuansanjuanp@gmail.com"

## 3. Funciones de la API

- `Entrez.email`: dirección del usuario. String sin espacios con una dirección de correo válida. Para más de 3 solicitudes/s se requiere una api_key.  
- `Entrez.einfo`: devuelve una lista de las bases de datos disponibles. Si se mete una base de datos como parámetro, por ejemplo, `info = Entrez.einfo(db='pubmed')`, proporciona información detallada de la base de datos (los campos disponibles)
- `Entrez.esearch`: realiza la búsqueda en una base de datos determinada, según una query. Devuelve los identificadores de los artículos (lista de UIDs). `handle = Entrez.esearch(db=db, term=term)` La query debe tener los caracteres especiales codificados en URL (' ' = +). Se usa HTTP POST para queries largas. Opcionalmente permite especificar un filtro para la búsqueda basado en la fecha.
- `Entrez.esummary`: obtiene los resúmenes de los artículos (*DocSum*) encontrados utilizando los identificadores (parámetro: id). Sin necesidad de recuperar todo el contenido de los artículos, obtiene la información clave.
- `Entrez.efetch`: obtener el contenido completo de los artículos (parámetro: id). Para visualizar el resultado tiene la función `.read()`

Como he comentado antes, *Entrez* permite almacenar temporalmente conjuntos de UID en un historial. A estos conjuntos se accede a través del servidor *Entrez History*, a partir de la Web se asigna a cada conjunto de UID una clave de consulta y un entorno web. Para utilizar y gestionar el servicio, en primer lugar se obtiene la lista de UIDs con la función `esearch` antes mencionada. Posteriormente, se carga la lista en el servidor del historial con `Entrez.epost`, permite cargar la lista de identificadores en un entorno web determiando (que se identifica con el parámetro `webenv`).  Para acceder a esos registros guardados en el historial, la función `esearch` permite determinar el `webenv` correspondiente como parámetro opcional. 


## 4. Código

La funcionalidad del siguiente código consiste en: 
- Obtener las bases de datos disponibles con esta API.
- Obtener información detallada de la base de datos de *PubMed*.
- Realizar una búsqueda de *n* artículos a través de una serie de palabras clave y una fecha para obtener los artículos que sean posteriores. 
- Obtener una lista con sus ID, un diccionario con sus ID, título, Autores, Fecha de publicación, código DOI y link a la página de *PubMed* del artículo. 
- Seleccionar uno de los artículos resultantes para imprimir su resumen por pantalla. 

In [4]:
def available_dbs():
    # Obtiene las bases de datos disponibles con la API de Entrez
    info = Entrez.einfo()
    record_info = Entrez.read(info)
    databases = record_info["DbList"]
    return databases

def pubmed_info(): 
    # Obtiene informacion detallada de la base de datos del PubMed
    info = Entrez.einfo(db="pubmed")
    record_info = Entrez.read(info)
    db_info = record_info['DbInfo']
    df_main = {
        "Dbname": db_info["DbName"],
        "Menuname": db_info["MenuName"],
        "Description": db_info["Description"],
        "Dbbuild": db_info["DbBuild"],
        "Count": db_info["Count"],
        "Lastupdate": db_info["LastUpdate"],
    }
    df_field_list = pd.DataFrame(db_info["FieldList"])
    return df_main, df_field_list

In [5]:
def search_articles(key_words, date, max_results=10):
    # Realiza la búsqueda de artículos en PubMed 
    term = f"{key_words}[Title/Abstract] AND {date}[PDAT]"
    search = Entrez.esearch(db="pubmed", term=term, retmax=max_results)
    record_search = Entrez.read(search)
    return record_search["IdList"]

def article_details(id_list):
    summary = Entrez.esummary(db="pubmed", id=",".join(id_list))
    record_summary = Entrez.read(summary)
    articles_info = []
    for article in record_summary:
        article_info = {
            "ID": article["Id"],
            "Título": article["Title"],
            "Autores": article["AuthorList"],
            "Fecha de publicación": article["PubDate"],
            "DOI": article.get("DOI", "No disponible"),
            "Link PubMed": f"https://www.ncbi.nlm.nih.gov/pubmed/{article['Id']}"
        }
        articles_info.append(article_info)
    return articles_info

def article_abstract(article_id):
    fetch = Entrez.efetch(db="pubmed", id=article_id, rettype="abstract", retmode="text")
    abstract = fetch.read()
    print(abstract)


In [6]:
# Obtener bases de datos disponibles
print("Bases de datos disponibles con Entrez:")
print(available_dbs())
print('-'*20)

# Obtener información detallada de PubMed
df_main, df_field_list= pubmed_info()
print('Información principal de PubMed:\n', df_main)
print('\nInformación de los campos  de la base de datos:\n')
df_field_list.head()

Bases de datos disponibles con Entrez:
['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'medgen', 'mesh', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'proteinclusters', 'pcassay', 'protfam', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'gtr']
--------------------
Información principal de PubMed:
 {'Dbname': 'pubmed', 'Menuname': 'PubMed', 'Description': 'PubMed bibliographic record', 'Dbbuild': 'Build-2024.02.10.23.59', 'Count': '36825504', 'Lastupdate': '2024/02/10 23:59'}

Información de los campos  de la base de datos:



,Name,FullName,Description,TermCount,IsDate,IsNumerical,SingleToken,Hierarchy,IsHidden
0,ALL,All Fields,All terms from all searchable fields,,N,N,N,N,N
1,UID,UID,Unique number assigned to publication,,N,Y,Y,N,Y
2,FILT,Filter,Limits the records,,N,N,Y,N,N
3,TITL,Title,Words in title of publication,,N,N,N,N,N
4,MESH,MeSH Terms,Medical Subject Headings assigned to publication,,N,N,Y,Y,N


In [7]:
# Realizar una búsqueda de artículos
key_words = ["cancer", "brain"]
date = "2023"
print(f"\nBúsqueda de artículos sobre '{key_words}' publicados a partir de {date}:")
id_list = search_articles(key_words, date)
print("IDs de los artículos encontrados:", id_list)


Búsqueda de artículos sobre '['cancer', 'brain']' publicados a partir de 2023:
IDs de los artículos encontrados: ['38328712', '38303306', '38260227', '38201564', '38187734', '38175350', '38149244', '38145439', '38142850', '38131219']


In [8]:
# Obtener información detallada de los artículos en un dataframe
articles_info = article_details(id_list)
print("\nInformación detallada de los artículos:")
df = pd.DataFrame(articles_info)
df


Información detallada de los artículos:


,ID,Título,Autores,Fecha de publicación,DOI,Link PubMed
0,38328712,Experimental models for cancer brain metastasis.,"[Liu Z, Dong S, Liu M, Liu Y, Ye Z, Zeng J, Ya...",2024 Jan,10.1016/j.cpt.2023.10.005,https://www.ncbi.nlm.nih.gov/pubmed/38328712
1,38303306,[A Case of Breast Cancer Brain Metastases Succ...,"[Hikino H, Otani A, Makino Y, Murata Y]",2023 Dec,No disponible,https://www.ncbi.nlm.nih.gov/pubmed/38303306
2,38260227,Effects of Ataxia-Telangiectasia Mutated Varia...,"[Floyd W, Carpenter D, Vaios E, Shenker R, Hen...",2024 Jan,10.1016/j.adro.2023.101320,https://www.ncbi.nlm.nih.gov/pubmed/38260227
3,38201564,Stereotactic Radiosurgery for Women Older than...,"[Upadhyay R, Klamer BG, Perlow HK, White JR, B...",2023 Dec 27,10.3390/cancers16010137,https://www.ncbi.nlm.nih.gov/pubmed/38201564
4,38187734,Discovery of novel brain permeable human ACSS2...,"[Esquea E, Ciraku L, Young RG, Merzy J, Talari...",2023 Dec 23,10.1101/2023.12.22.573073,https://www.ncbi.nlm.nih.gov/pubmed/38187734
5,38175350,Breast Cancer Brain Metastases: Achilles' Heel...,"[Ferraro E, Seidman AD]",2023,10.1007/978-3-031-33602-7_11,https://www.ncbi.nlm.nih.gov/pubmed/38175350
6,38149244,Unlocking molecular mechanisms and identifying...,"[Najjary S, de Koning W, Kros JM, Mustafa DAM]",2023,10.3389/fimmu.2023.1305644,https://www.ncbi.nlm.nih.gov/pubmed/38149244
7,38145439,Air quality and cancer risk in the All of Us R...,"[Craver A, Luo J, Kibriya MG, Randorf N, Bahl ...",2023 Dec 25,10.1007/s10552-023-01823-7,https://www.ncbi.nlm.nih.gov/pubmed/38145439
8,38142850,Microtubule destabilising activity of selected...,"[Perużyńska M, Birger R, Piotrowska K, Kwiecie...",2024 Feb 5,10.1016/j.ejphar.2023.176308,https://www.ncbi.nlm.nih.gov/pubmed/38142850
9,38131219,Evaluation of brain metastasis edema in breast...,"[Stojkova M, Behme D, Barajas Ordonez F, Chris...",2023 Dec 22,10.1177/19714009231224443,https://www.ncbi.nlm.nih.gov/pubmed/38131219


In [9]:
# Seleccionar un artículo para imprimir su resumen
if id_list:
    article_id = id_list[0]
    print("\nResumen del primer artículo encontrado:")
    article_abstract(article_id)
else:
    print("\nNo se encontraron artículos.")


Resumen del primer artículo encontrado:
1. Cancer Pathog Ther. 2023 Oct 29;2(1):15-23. doi: 10.1016/j.cpt.2023.10.005. 
eCollection 2024 Jan.

Experimental models for cancer brain metastasis.

Liu Z(1), Dong S(2), Liu M(1), Liu Y(1), Ye Z(1), Zeng J(3), Yao M(1).

Author information:
(1)Guangzhou Institute of Respiratory Disease, The First Affiliated Hospital of 
Guangzhou Medical University, China State Key Laboratory of Respiratory Disease, 
Guangzhou, Guangdong 510182, China.
(2)Department of Medical Genetics and Cell Biology, GMU-GIBH Joint School of 
Life Sciences, Guangzhou Medical University, Guangzhou, Guangdong 511436, China.
(3)Department of Microbiology, Immunology, and Cancer Biology, University of 
Virginia Health System, Charlottesville, VA 22908, USA.

Brain metastases are a leading cause of cancer-related mortality. However, 
progress in their treatment has been limited over the past decade, due to an 
incomplete understanding of the underlying biological mechanisms. E

## 5. Conclusión

La elección de utilizar la API de *PubMed* para obtener datos surgió de mi especial interés en el ámbito biomédico y mi experiencia previa con la plataforma para la documentación de otros proyectos de investigación. Sin embargo, no estaba al tanto de la posibilidad de acceder a los datos a través de una API. Esta decisión no solo ha ampliado mis habilidades técnicas, sino que también me ha revelado la utilidad y relevancia de esta herramienta en la bioinformática. Me ha resultado muy enriquecedor este proyecto ya que no conocía las técncias de scraping en general antes de realizar la asignatura, y además, ha sido una grata sorpresa que el acceso a estos bancos de información sea público, creo que es crucial para el avance en el sector de la investigación. 